## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Hyperparameters

In [4]:
# Tokenizer arguments
max_length = 1024

# model arguments
max_new_tokens=1000

# mixed precision
dtype = torch.bfloat16

# quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=dtype,
    bnb_4bit_quant_type="nf4"
)

## Model

In [5]:
# Model List

# gemma variants
# "google/gemma-7b-it" // downloaded
# "PathFinderKR/waktaverse-gemma-ko-7b-it"
# "beomi/gemma-ko-7b"

# llama2 variants
# "meta-llama/Llama-2-7b-chat-hf" // downloaded
# "PathFinderKR/waktaverse-Llama-2-ko-7b-it"
# "beomi/KoAlpaca-Polyglot-5.8B" // downloaded
# "beomi/open-llama-2-ko-7b"

# solar variants
# "chihoonlee10/T3Q-ko-solar-dpo-v4.0"

In [6]:
model_id = "PathFinderKR/waktaverse-gemma-ko-7b-it"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation="flash_attention_2",
    torch_dtype=dtype,
    quantization_config=quantization_config
)

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Running the model on a single GPU

In [8]:
# Chat Template
def generate_response(prompt):
    messages = [{"role": "user", "content": prompt}]
    chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer.encode(chat, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=input_ids.to(model.device), max_new_tokens=max_new_tokens)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [9]:
#prompt = "Write me a poem about Machine Learning."
#prompt = "머신러닝에 대한 시를 써주세요."

In [10]:
prompt = "ppo 알고리즘에 대해 설명해줘"

In [11]:
response = generate_response(prompt)
print(response)

user
ppo 알고리즘에 대해 설명해줘
model
**Policy-Based Optimization (PPO)**는 강화 학습 알고리즘으로, 에이전트가 정책을 추정하는 과정에서 사용됩니다. 이 알고리즘은 기존 정책을 기반으로 새로운 정책을 생성하고, 이를 사용하여 에이전트가 더 많은 보상을 받도록 최적화합니다.

**주요 특징:**

* **정책 추정:** PPO는 기존 정책을 기반으로 새로운 정책을 생성합니다.
* **정책 평가:** 새로운 정책의 성능을 평가하고, 이를 기존 정책과 비교합니다.
* **정책 개선:** 새로운 정책의 성능을 기반으로 기존 정책을 개선합니다.
* **반복:** 위 과정을 반복하여 최적의 정책을 찾습니다.

**주요 단계:**

1. **정책 생성:** 기존 정책을 기반으로 새로운 정책을 생성합니다.
2. **정책 평가:** 새로운 정책의 성능을 평가하고, 이를 기존 정책과 비교합니다.
3. **정책 개선:** 새로운 정책의 성능을 기반으로 기존 정책을 개선합니다.
4. **정책 수동:** 최적의 정책을 찾으면, 이를 사용하여 에이전트가 더 많은 보상을 받도록 최적화합니다.

**장점:**

* **정책 추정:** PPO는 기존 정책을 기반으로 새로운 정책을 생성하므로, 기존 정책의 장점을 유지하면서도 개선할 수 있습니다.
* **정책 평가:** PPO는 정책의 성능을 평가하는 과정에서, 기존 정책의 확률 분포를 사용하여 정책의 성능을 평가할 수 있습니다.
* **정책 개선:** PPO는 새로운 정책의 성능을 기반으로 기존 정책을 개선할 수 있습니다.

**단점:**

* **정책 수동:** PPO는 최적의 정책을 찾는 과정에서, 이를 수동으로 수동으로 선택하는 것이 필요합니다.
* **정책 과대:** PPO는 기존 정책의 확률 분포를 사용하여 정책의 성능을 평가하므로, 기존 정책의 과대 분포에 의해 정책의 성능에 영향을 미칠 수 있습니다.
